## Bienvenidos

<br />
<br />

# TradeGestorDEMO 
### BingX Version

<br />

*SECUENCIA LOGICA:*

- 1. Obtener Volumen de la cuenta (API)
- 2. Solicitar al usuario el riesgo por posición
- 3. Solicitar la dirección del trade (LONG/SHORT)
- 4. Solicitar la cantidad de entradas por posición
- 5. Solicitar que defina estado entrada (COLOCAR/ANULAR)
- 6. Solicitar TARGETS (entrada y SL) para entradas Limit
- 7. Ejecutar entrada o dejar en pendiente

In [1]:
# VERSION 1: Estructura de las variables principales
if False:
    vol_cta = input('VOLUMEN DE LA CUENTA: ')#TODO: API
    riesgo_posicion = input('RIESGO POR POSICION: ')
    direccion_trade = input('LONG/SHORT: ')#    -> default:
    n_entradas = input('CANTIDAD DE ENTRADAS: ')#    -> default: 1
    estado_entradas = [True, True, False]# COLOCAR/ANULAR # 
    target_entradas = [(0,27877),(27900,28150)] #    -> default: (0,)
    #DESCARGAR TXT
    #AHORA PUEDO PROMEDIAR LAS ENTRADAS PARA CALCULAR EL APALANCAMIENTO (dejar un margen por si la actualización es muy grotesca-)#TODO: if 0 -> API CALL
    ejecutar_entradas = [1,1]#    -> default: Ejecutar

In [1]:
import os

if os.path.isfile('.env'):
    print('El archivo .env existe en el directorio de trabajo')
else:
    print('El archivo .env no existe en el directorio de trabajo')

El archivo .env existe en el directorio de trabajo


In [12]:
# VERSION 2: Definiendo los valores de forma genérica
import numpy as np

def api_call_price():
    return 27123

vol_cta = 500
riesgo_op = 5

direccion_trade = 1 #LONG

n_unidades = 5

estado_entradas = []
for i in range(n_unidades):
    # ingresar bool
    if i < 2:
        estado_entradas.append(1)
    else:
        estado_entradas.append(0)

target_entradas = []
for estado in estado_entradas:
    if estado:
        entrada = int(input('entrada: '))#default(0): for market value, or exact value
        sl = int(input('stop loss: '))# exact value or value with "%" at the end
        
        if not entrada:
            entrada = api_call_price()
        
        target_entradas.append((entrada, sl))


#AHORA PUEDO PROMEDIAR LAS ENTRADAS PARA CALCULAR EL APALANCAMIENTO (dejar un margen por si la actualización es muy grotesca-)#TODO: if 0 -> API CALL
#1. promediar entradas
entrada_promedio = np.mean([entrada for entrada, _ in target_entradas])#TODO: error acá tenemos 5 entradas y solo 2 son a colocar
#2. Obtener el stop loss más alejado 
if direccion_trade:
    worst_sl = np.min([sl for _, sl in target_entradas])
else:
    worst_sl = np.max([sl for _, sl in target_entradas])
#3. medir el apalancamiento para estar lo más cerca posible del stop loss más alejado
def apalancamiento(precio_entrada):
    """
    precio_entrada / apalancamiento = distancia entre el precio de entrada y el stop loss más alejado
    Para trabajar con margen de seguridad le quitamos una unidad al apalancamiento
    """
    if worst_sl < precio_entrada: # -> LONG
        dist = precio_entrada - worst_sl
    if worst_sl > precio_entrada:# -> SHORT
        dist = worst_sl - precio_entrada
    apalancamiento = int(precio_entrada / dist) -1
    return apalancamiento
apal_x = apalancamiento(entrada_promedio)


def ejecutar():
    """
    Función que ejecuta el trade
    """
    if input('Ejecutar?: '):
        print('Ejecutando...')
        return True
    else:
        print('Entrada pendiente')
        return False

for entrada in target_entradas:
    if ejecutar():
        print('Entrada ejecutada')
    else:
        print('Entrada pendiente')



Ejecutando...
Entrada ejecutada
Ejecutando...
Entrada ejecutada


In [1]:
# VERSION 3: Resolviendo el problema de la entrada promedio (BACKUP: 15:20hs)
import numpy as np

def api_call_price():
    return 27123

vol_cta = 500
riesgo_op = 5#%
direccion_trade = 1 #LONG

n_unidades = 5

estado_entradas = []
for i in range(n_unidades):
    # ingresar bool
    if i < 2:
        estado_entradas.append(1)
    else:
        estado_entradas.append(0)

target_entradas = []
for estado in estado_entradas:
    if estado:
        entrada = int(input('entrada: '))#default(0): for market value, or exact value
        sl = int(input('stop loss: '))# exact value or value with "%" at the end
        
        if not entrada:
            entrada = api_call_price()
        
        target_entradas.append((entrada, sl))


# DIMENSIONAMIENTO DEL TRADE
#1. promediar entradas
entradas = [entrada for entrada, _ in target_entradas]
entrada_promedio = np.mean(entradas)
#2. Obtener el stop loss más alejado
sls = [sl for _, sl in target_entradas]
if direccion_trade:
    worst_sl = np.min(sls)
else:
    worst_sl = np.max(sls)
#3. medir el apalancamiento para estar lo más cerca posible del stop loss más alejado
def apalancamiento(precio_entrada):
    """
    precio_entrada / apalancamiento = distancia entre el precio de entrada y el stop loss más alejado
    Para trabajar con margen de seguridad le quitamos una unidad al apalancamiento
    """
    if worst_sl < precio_entrada: # -> LONG
        dist = precio_entrada - worst_sl
    if worst_sl > precio_entrada:# -> SHORT
        dist = worst_sl - precio_entrada
    apalancamiento = int(precio_entrada / dist) -1
    return apalancamiento
apal_x = apalancamiento(entrada_promedio)


# CALCULO DE LA POSICION
vol_op = vol_cta * riesgo_op /100
vol_unidad = vol_op / n_unidades
qty_entradas = [vol_unidad / entrada_i * apal_x for entrada_i in entradas]

## EJECUCION DE LA POSICION
def ejecutar():
    """
    Función que ejecuta el trade
    """
    if input('Ejecutar?: '):
        print('Ejecutando...')
        return True
    else:
        print('Entrada pendiente')
        return False

for entrada in target_entradas:
    if ejecutar():
        print('Entrada ejecutada')
    else:
        print('Entrada pendiente')


In [2]:
# VERSION 4 PARTE 1: Empezamos a implementar la API
import numpy as np
import os
from dotenv import load_dotenv
import requests
import time
from http.client import HTTPException
import hmac
import hashlib



load_dotenv()

API_KEY = os.environ['API_KEY'] 
SECRET_KEY = os.environ['SECRET_KEY'] 

URL = 'https://open-api.bingx.com'

BINGX_ERRORS = {
    100001: "signature verification failed",
    100500: "Internal system error",
    80001: "request failed",
    80012: "service unavailable",
    80014: "Invalid parameter",
    80016: "Order does not exist",
    80017: "position does not exist"
}

services = {'GET_1':'/openApi/swap/v2/quote/contracts',
            'GET_2':'/openApi/swap/v2/user/balance',
            'GET_3':'/openApi/swap/v2/quote/price',
}


def generate_signature(secret_key, query_params):
    """Generar firma encriptada"""
    signature = hmac.new(
        key = secret_key.encode('utf-8'), 
        msg = query_params.encode('utf-8'), 
        digestmod = hashlib.sha256
        ).hexdigest()
    return signature

def api_request(service, method='GET', query_params=None, sign=False):
    """docstring"""
    ROOT_URL = URL
    url = f'{ROOT_URL}{service}'
    header = {
        'Content-Type': 'application/json',
        'X-BX-APIKEY': API_KEY,
    }

    timestamp = int(time.time() *1000)
    params = f'timestamp={timestamp}'
    if query_params:
        params += f'&{query_params}'
    if sign:
        signature = generate_signature(SECRET_KEY, params)
        params += f'&signature={signature}'
    
    url += f'?{params}'
    if method == 'GET':
        response = requests.get(url, headers=header)
    else:
        response = requests.post(url, headers=header)

    if response.status_code != 200:
        print (f'status_code: {response.status_code}')
        print (f'message: {response.text}')
        raise HTTPException()

    response = response.json()
    if 'success' in response and not response.get('success'):
        print ('status_code=400')
        print (f'detail={BINGX_ERRORS.get(response.get("code"))}')
        raise HTTPException()

    return response

# contracts = api_request(service='/openApi/swap/v2/quote/contracts', query_params='BTCUSD')
# account_balance = api_request('/openApi/swap/v2/user/balance', method='GET', sign=True)

In [8]:
## SEGUNDA CAPA DE LA API

def get_account_balance():
    account_balance = api_request('/openApi/swap/v2/user/balance', method='GET', sign=True)
    return np.floor(float(account_balance['data']['balance']['balance']))

def price_BTC():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=BTC-USDT')['data']['price'])
def price_ETH():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=ETH-USDT')['data']['price'])

In [1]:
# INFORMACION SOBRE LOS CONTRATOS

"""
Información relacionada a BingX Perpetual Futures

193 tipos de contratos

response.keys(): dict_keys(['code', 'msg', 'data'])



Información del contrato:
{'contractId': '100',
    'symbol': 'BTC-USDT',
    'size': '0.0001',
    'quantityPrecision': 4,
    'pricePrecision': 1,
    'feeRate': 0.0005,
    'tradeMinLimit': 1,
    'maxLongLeverage': 150,
    'maxShortLeverage': 150,
    'currency': 'USDT',
    'asset': 'BTC',
    'status': 1
}

{'contractId': '101',
    'symbol': 'ETH-USDT',
    'size': '0.01',
    'quantityPrecision': 2,
    'pricePrecision': 2,
    'feeRate': 0.0005,
    'tradeMinLimit': 1,
    'maxLongLeverage': 125,
    'maxShortLeverage': 125,
    'currency': 'USDT',
    'asset': 'ETH',
    'status': 1}

"""



"\nInformación relacionada a BingX Perpetual Futures\n\n193 tipos de contratos\n\nresponse.keys(): dict_keys(['code', 'msg', 'data'])\n\n\n\nInformación del contrato:\n{'contractId': '100',\n    'symbol': 'BTC-USDT',\n    'size': '0.0001',\n    'quantityPrecision': 4,\n    'pricePrecision': 1,\n    'feeRate': 0.0005,\n    'tradeMinLimit': 1,\n    'maxLongLeverage': 150,\n    'maxShortLeverage': 150,\n    'currency': 'USDT',\n    'asset': 'BTC',\n    'status': 1\n}\n\n{'contractId': '101',\n    'symbol': 'ETH-USDT',\n    'size': '0.01',\n    'quantityPrecision': 2,\n    'pricePrecision': 2,\n    'feeRate': 0.0005,\n    'tradeMinLimit': 1,\n    'maxLongLeverage': 125,\n    'maxShortLeverage': 125,\n    'currency': 'USDT',\n    'asset': 'ETH',\n    'status': 1}\n\n"

In [4]:
# VERSION 4 PARTE 2.1

contract = {'contractId': '100',
    'symbol': 'BTC-USDT',
    'size': '0.0001',
    'quantityPrecision': 4,
    'pricePrecision': 1,
    'feeRate': 0.0005,
    'tradeMinLimit': 1,
    'maxLongLeverage': 150,
    'maxShortLeverage': 150,
    'currency': 'USDT',
    'asset': 'BTC',
    'status': 1
}


def apalancamiento(precio_entrada):
    """
    precio_entrada / apalancamiento = distancia entre el precio de entrada y el stop loss más alejado
    Para trabajar con margen de seguridad le quitamos una unidad al apalancamiento
    """
    dist = abs(precio_entrada - worst_sl)
    apalancamiento = int(precio_entrada / dist) -2
    if direccion_trade == 'LONG':
        precio_liquidacion = precio_entrada *(1 - 1/apalancamiento)
    elif direccion_trade == 'SHORT':
        precio_liquidacion = precio_entrada *(1 + 1/apalancamiento)
    return apalancamiento, precio_liquidacion


vol_cta = get_account_balance()

In [23]:
# VERSION 4 PARTE 2.2

from tools.ingresar_datos import ingreso_bool, ingreso_bool_personalizado, ingreso_entero, entero_o_porcentual

riesgo_op = 5#%
print ('Direccion del trade:')
direccion_trade = ingreso_bool_personalizado('LONG', 'SHORT')
n_unidades = 5
if n_unidades <= 0 or n_unidades > 5:
    raise ValueError('La posición solo admite hasta 5 entradas')

# CALCULO DE LA POSICION
# vol_cta *= 10 #  <-- preapalancamiento
vol_op = vol_cta * riesgo_op /100
vol_unidad = vol_op / n_unidades

print ("""
DATOS DE LA CUENTA
    volumen cuenta  = {}

DATOS DE LA POSICION
    riesgo posicion = {}%
    direccion trade = {}
    nº entradas = {}
    volumen operacion = {}
    volumen por entrada = {}
        """.format(vol_cta, riesgo_op, direccion_trade, n_unidades, vol_op, vol_unidad))

estado_entradas = []
for i in range(n_unidades):
    estado_entradas.append(ingreso_bool(f'Colocar entrada {i+1}?'))
print (f'Declaración de entrada: {estado_entradas}')


target_entradas = []
benchmark = price_BTC()
for estado in estado_entradas:
    if estado:
        entrada = ingreso_bool_personalizado('MARKET', 'LIMIT', default='MARKET')
        if entrada == 'LIMIT':
            entrada, pct = entero_o_porcentual(f'BTC: {benchmark}  |  Ingrese precio de entrada: ')
            if pct and direccion_trade=='LONG':
                entrada = benchmark *(1-entrada)
            elif pct and direccion_trade=='SHORT':
                entrada = benchmark *(1+entrada)
            elif not entrada:
                raise ValueError('No se puede ingresar un orden LIMIT sin precio entrada')
        elif entrada == 'MARKET':
            entrada = benchmark
        else:
            raise ValueError('No se puede colocar una entrada vacia')

        sl, pct = entero_o_porcentual(f'BTC {benchmark}  |  Ingrese target StopLoss: ')
        if pct and direccion_trade=='LONG':
            sl = benchmark * (1 - sl)
        elif pct and direccion_trade=='SHORT':
            sl = benchmark * (1 + sl)
        elif not sl:
            raise ValueError('No se puede puede ingresar una orden LIMIT sin target StopLoss')

        target_entradas.append((entrada, sl))

        for entrada, sl in target_entradas:
            if direccion_trade == 'LONG':
                if entrada <= sl:
                    raise ValueError('En un trade LONG, la entrada no puede ser menor que el StopLoss')
            elif direccion_trade == 'SHORT':
                if entrada >= sl:
                    raise ValueError('En un trade SHORT, la entrada no puede ser mayor que el StopLoss')


# DIMENSIONAMIENTO DEL TRADE
#1. promediar entradas
entradas = [entrada for entrada, _ in target_entradas]
entrada_promedio = np.mean(entradas)
#2. Obtener el stop loss más alejado
sls = [sl for _, sl in target_entradas]
if direccion_trade == 'LONG':
    worst_sl = np.min(sls)
elif direccion_trade == 'SHORT':
    worst_sl = np.max(sls)
#3. medir el apalancamiento para estar lo más cerca posible del stop loss más alejado
apal_x, precio_liq = apalancamiento(entrada_promedio)

qty_entradas = [vol_unidad / entrada_i * apal_x for entrada_i in entradas]

import sys
sys.exit(0)

## EJECUCION DE LA POSICION
def ejecutar():
    """
    Función que ejecuta el trade
    """
    if input('Ejecutar?: '):
        print('Ejecutando...')
        return True
    else:
        print('Entrada pendiente')
        return False

for entrada in target_entradas:
    if ejecutar():
        print('Entrada ejecutada')
    else:
        print('Entrada pendiente')


Direccion del trade:
< 1: LONG >    < 0: SHORT >


DATOS DE LA CUENTA
    volumen cuenta  = 50.0

DATOS DE LA POSICION
    riesgo posicion = 5%
    direccion trade = LONG
    nº entradas = 5
    volumen operacion = 2.5
    volumen por entrada = 0.5
        
Colocar entrada 1?
< 1: Si >    < 0: No >

Colocar entrada 2?
< 1: Si >    < 0: No >

Colocar entrada 3?
< 1: Si >    < 0: No >

Colocar entrada 4?
< 1: Si >    < 0: No >

Colocar entrada 5?
< 1: Si >    < 0: No >

Declaración de entrada: [True, True, False, False, False]
< 1: MARKET >    < 0: LIMIT >

BTC 27738.0  |  Ingrese target limit: 
< 1: MARKET >    < 0: LIMIT >

BTC: 27738.0  |  Ingrese precio de entrada: 


ValueError: No se puede ingresar un orden LIMIT sin precio entrada

In [1]:
#  VERSION 5: LOGIC

import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import time
from http.client import HTTPException
import hmac
import hashlib
from rich.console import Console
from rich.table import Table
from datetime import datetime
import locale

locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")


load_dotenv()

API_KEY = os.environ['API_KEY'] 
SECRET_KEY = os.environ['SECRET_KEY'] 

URL = 'https://open-api.bingx.com'

BINGX_ERRORS = {
    100001: "signature verification failed",
    100500: "Internal system error",
    80001: "request failed",
    80012: "service unavailable",
    80014: "Invalid parameter",
    80016: "Order does not exist",
    80017: "position does not exist"
}

services = {'GET_1':'/openApi/swap/v2/quote/contracts',
            'GET_2':'/openApi/swap/v2/user/balance',
            'GET_3':'/openApi/swap/v2/quote/price',
}


def generate_signature(secret_key, query_params):
    """Generar firma encriptada"""
    signature = hmac.new(
        key = secret_key.encode('utf-8'), 
        msg = query_params.encode('utf-8'), 
        digestmod = hashlib.sha256
        ).hexdigest()
    return signature

def api_request(service, method='GET', query_params=None, sign=False):
    """docstring"""
    ROOT_URL = URL
    url = f'{ROOT_URL}{service}'
    header = {
        'Content-Type': 'application/json',
        'X-BX-APIKEY': API_KEY,
    }

    timestamp = int(time.time() *1000)
    params = f'timestamp={timestamp}'
    if query_params:
        params += f'&{query_params}'
    if sign:
        signature = generate_signature(SECRET_KEY, params)
        params += f'&signature={signature}'
    
    url += f'?{params}'
    if method == 'GET':
        response = requests.get(url, headers=header)
    else:
        response = requests.post(url, headers=header)

    if response.status_code != 200:
        print (f'status_code: {response.status_code}')
        print (f'message: {response.text}')
        raise HTTPException()

    response = response.json()
    if 'success' in response and not response.get('success'):
        print ('status_code=400')
        print (f'detail={BINGX_ERRORS.get(response.get("code"))}')
        raise HTTPException()

    return response

# contracts = api_request(service='/openApi/swap/v2/quote/contracts', query_params='BTCUSD')
# account_balance = api_request('/openApi/swap/v2/user/balance', method='GET', sign=True)


## SEGUNDA CAPA DE LA API

def get_account_balance():
    account_balance = api_request('/openApi/swap/v2/user/balance', method='GET', sign=True)
    return np.floor(float(account_balance['data']['balance']['balance']))

def price_BTC():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=BTC-USDT')['data']['price'])
def price_ETH():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=ETH-USDT')['data']['price'])
def price_XRP():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=XRP-USDT')['data']['price'])






## FUNCIONES DEL MAIN


def apalancamiento(precio_entrada, worst_sl, direccion_trade):
    """
    precio_entrada / apalancamiento = distancia entre el precio de entrada y el stop loss más alejado
    Para trabajar con margen de seguridad le quitamos una unidad al apalancamiento
    """
    dist = abs(precio_entrada - worst_sl)
    apalancamiento = int(precio_entrada / dist) -2
    if direccion_trade == 'LONG':
        precio_liquidacion = precio_entrada *(1 - 1/apalancamiento)
    elif direccion_trade == 'SHORT':
        precio_liquidacion = precio_entrada *(1 + 1/apalancamiento)
    return apalancamiento, precio_liquidacion

def obtener_contrato(par: str) -> pd.Series:
    contracts = api_request(services['GET_1'])
    contracts = pd.DataFrame(contracts['data'])
    if par == 'XRP':
        XRP_mask = contracts['symbol'].apply(lambda x: x.startswith('XRP')).values
        XRP_contract = contracts.loc[XRP_mask]
        return XRP_contract.iloc[0]
    elif par == 'BTC':
        BTC_mask = contracts['symbol'].apply(lambda x: x.startswith('BTC')).values
        BTC_contracts = contracts.loc[BTC_mask]
        BTC_contract = BTC_contracts.iloc[0]
        BTCD_contract = BTC_contracts.iloc[1]
        return BTC_contract
    return

In [68]:
# VERSION 5 MAIN
if __name__ == '__main__':
    from tools.ingresar_datos import ingreso_bool, ingreso_bool_personalizado, entero_o_porcentual
    from itertools import count

    print ('''
    ============================================================
            BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
    ============================================================
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

    ''')

    par = ingreso_bool_personalizado('BTC', 'XRP')
    if not par:
        exit()
    contract = obtener_contrato(par)
    qty_precision = contract['quantityPrecision']
    price_precision = contract['pricePrecision']

    vol_cta = get_account_balance()
    riesgo_op = 5#%
    print ('Direccion del trade:')
    direccion_trade = ingreso_bool_personalizado('LONG', 'SHORT')
    n_unidades = 2
    if n_unidades <= 0 or n_unidades > 5:
        raise ValueError('La posición solo admite hasta 5 entradas')

    # CALCULO DE LA POSICION
    # vol_cta *= 10 #  <-- preapalancamiento
    vol_op = vol_cta * riesgo_op /100
    vol_unidad = vol_op / n_unidades

    display_data_inicial = """
>>    DATOS DE LA CUENTA
        - volumen cuenta  = {}

>>    DATOS DE LA POSICION
        - riesgo posicion = {}%
        - direccion trade = {}
        - nº entradas = {}
        - volumen operacion = {}
        - volumen por entrada = {}

        """.format(vol_cta, riesgo_op, direccion_trade, n_unidades, vol_op, round(vol_unidad,2))
    print (display_data_inicial)

    print ('''
    ============================================================
                    MODULO DE DIVERSIFICACION
    ============================================================
    
    ''')
    estado_entradas = []
    for i in range(n_unidades):
        estado_entradas.append(ingreso_bool(f'Colocar entrada {i+1}?'))
    print (f'Entradas COLOCADAS: {sum(estado_entradas)}  |  Entradas ANULADAS: {len(estado_entradas)-sum(estado_entradas)}')


    target_entradas = []
    print ('\nCargando precio...')
    if par == 'BTC':
        benchmark = price_BTC()
    elif par == 'XRP':
        benchmark = price_XRP()
    print ('continuando\n')
    display_estado = """

    ============================================================
                            ENTRADA Nº {}
    ============================================================
    Precio de referencia {} = {}
    """
    entrada_count = count(1)
    for estado in estado_entradas:
        if estado:
            print (display_estado.format(next(entrada_count), par, benchmark))
            # DETERMINACION DE LA ENTRADA
            entrada = ingreso_bool_personalizado('MARKET', 'LIMIT', default='MARKET')
            if entrada == 'LIMIT':
                entrada, pct = entero_o_porcentual('>>  PRECIO DE ENTRADA (Vacío significa precio MARKET) ')
                if pct and direccion_trade=='LONG':
                    entrada = benchmark *(1-entrada)
                elif pct and direccion_trade=='SHORT':
                    entrada = benchmark *(1+entrada)
                elif not entrada:
                    print ('Ingresando orden LIMIT a precio MARKET')
                    entrada = benchmark
                    # raise ValueError('No se puede ingresar un orden LIMIT sin precio entrada')
            elif entrada == 'MARKET':
                entrada = benchmark
            else:
                raise ValueError('No se puede colocar una entrada vacia')
            # DETERMINACION DEL STOPLOSS
            sl, pct = entero_o_porcentual('>>  STOPLOSS ')
            if pct and direccion_trade=='LONG':
                sl = benchmark * (1 - sl)
            elif pct and direccion_trade=='SHORT':
                sl = benchmark * (1 + sl)
            elif not sl:#TODO meter la SL dentro de un bucle while o agregar la función de chance
                raise ValueError('No se puede puede ingresar una orden LIMIT sin target StopLoss')

            target_entradas.append((entrada, sl))
            # VERIFICACION
            for entrada, sl in target_entradas:
                if direccion_trade == 'LONG':
                    if entrada <= sl:
                        raise ValueError('En un trade LONG, la entrada no puede ser menor que el StopLoss')
                elif direccion_trade == 'SHORT':
                    if entrada >= sl:
                        raise ValueError('En un trade SHORT, la entrada no puede ser mayor que el StopLoss')


    # DIMENSIONAMIENTO DEL TRADE
    print ('\nCalculando posición...')
    #1. promediar entradas
    entradas = [entrada for entrada, _ in target_entradas]
    entrada_promedio = np.mean(entradas)
    #2. Obtener el stop loss más alejado
    sls = [sl for _, sl in target_entradas]
    if direccion_trade == 'LONG':
        worst_sl = np.min(sls)
    elif direccion_trade == 'SHORT':
        worst_sl = np.max(sls)
    #3. medir el apalancamiento para estar lo más cerca posible del stop loss más alejado, con un margen hardcoded de 2
    apal_x, precio_liq = apalancamiento(entrada_promedio, worst_sl, direccion_trade)

    qty_entradas = [round(vol_unidad/abs(x[0] - x[1]),qty_precision) for x in target_entradas]




    display_resultado = """
    
    ============================================================
                            RESULTADOS
    ============================================================
    
    """


    # IMPRESION DE RESULTADOS
    display_resultados =f"""
>>  DESCRIPCION TRADE {direccion_trade}
        - Precio de referencia = {benchmark}
        - Entrada promedio = {round(entrada_promedio, price_precision)}
        - StopLoss más alejado = {round(worst_sl, price_precision)}
        - Apalancamiento = {apal_x}
        - Precio de liquidación = {round(precio_liq, price_precision)}
        - Cantidad por entrada = {[round(qty_e, qty_precision) for qty_e in qty_entradas]}
        - Pérdidas peor escenario = {sum([precio*cantidad/apal_x for precio, cantidad in zip(entradas, qty_entradas)])}
        """
    print (display_resultados)

    # RESUMEN ENTRADAS
    console = Console()
    table = Table(title="INFORMACION POSICION: {} | PAR: {}".format(direccion_trade, par))

    table.add_column("Nº", justify="right", style="cyan", no_wrap=True)
    table.add_column("CONDICION", style="cyan")
    table.add_column("ENTRADA", justify="right", style="cyan")
    table.add_column("STOPLOSS", justify="right", style="cyan")
    table.add_column("CANTIDAD", justify="right", style="cyan")
    table.add_column("RIESGO", justify="right", style="cyan")

    def generar_rows(n_entradas, estado_entradas, entradas, sls, qty_entradas):
        id = [str(i) for i in range(1, n_entradas+1)]
        estado_entradas = ['Calculada' if e else 'Omitida' for e in estado_entradas]
        entradas = [entradas[i] if i < len(entradas) else 0.0 for i in range(n_entradas)]
        sls = [sls[i] if i < len(sls) else 0.0 for i in range(n_entradas)]
        qty_entradas = [qty_entradas[i] if i < len(qty_entradas) else 0.0 for i in range(n_entradas)]
        riesgo = [(entradas[i] - sls[i])*qty_entradas[i] for i in range(n_entradas)]

        entradas = [str(round(elemento, price_precision))for elemento in entradas]
        sls = [str(round(elemento, price_precision))for elemento in sls]
        qty_entradas = [str(round(elemento, qty_precision))for elemento in qty_entradas]
        riesgo = [str(round(elemento, 1))for elemento in riesgo]
        return id, estado_entradas, entradas, sls, qty_entradas, riesgo

    row_1, row_2, row_3, row_4, row_5, row_6 = generar_rows(n_unidades, estado_entradas, entradas, sls, qty_entradas)

    for i in range(n_unidades):
        table.add_row(row_1[i], row_2[i], row_3[i], row_4[i], row_5[i], row_6[i])
    console.print(table)
    


    ######      ######      ######
    ##  GUARDAR POSICION EN TXT
    ######      ######      ######


    file_location = '\\registro\\'
    # contador = count(1)#TODO
    fecha_actual = datetime.now()
    nombre_mes = fecha_actual.strftime("%B").title()
    hora = '\nTrade calculado el día {} a las {}'.format(fecha_actual.strftime("%d de {mes} de %Y").format(mes=nombre_mes), fecha_actual.strftime("%H:%M:%S"))
    file_name = f'{direccion_trade}_{par}_{fecha_actual.strftime("%d_{}").format(nombre_mes)}.txt'#next(contador)

    file = file_location + file_name
    
    if not os.path.exists(file):
        f = open(file, 'w')
        f.close(file)
    with open(file, 'w') as f:
        f.write(display_data_inicial)
        f.write(display_resultados)
        f.write(str(table))
        f.write(hora)


    ### OBTENEER LOS VALORES DEL RESULTADO INDIVIDUAL DE LAS ENTRADAS Y DEL TRADE EN GENERAL

    import sys
    sys.exit(0)

    ## EJECUCION DE LA POSICION
    def ejecutar():
        """
        Función que ejecuta el trade
        """
        if input('Ejecutar?: '):
            print('Ejecutando...')
            return True
        else:
            print('Entrada pendiente')
            return False

    for entrada in target_entradas:
        if ejecutar():
            print('Entrada ejecutada')
        else:
            print('Entrada pendiente')



            BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

    
< 1: BTC >    < 0: XRP >

Direccion del trade:
< 1: LONG >    < 0: SHORT >


>>    DATOS DE LA CUENTA
        - volumen cuenta  = 50.0

>>    DATOS DE LA POSICION
        - riesgo posicion = 5%
        - direccion trade = LONG
        - nº entradas = 2
        - volumen operacion = 2.5
        - volumen por entrada = 1.25

        

                    MODULO DE DIVERSIFICACION
    
    
Colocar entrada 1?
< 1: Si >    < 0: No >

Colocar entrada 2?
< 1: Si >    < 0: No >

Entradas COLOCADAS: 2  |  Entradas ANULADAS: 0

Cargando precio...
continuando



                            ENTRADA Nº 1
    Precio de referencia BTC = 27043.5
    
< 1: MARKET >    < 0: LIMIT >
>>  STOPLOSS (Vacío significa 0)


                            ENTRADA Nº 2
    Precio de referencia BTC = 27043.5
    
< 1: MARKET >    < 0: LIMIT >

>>  PRECIO DE ENTRADA (

                   INFORMACION POSICION: LONG | PAR: BTC                    
┏━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ ENTRADA Nº ┃ CONDICION ┃ ENTRADA ┃ STOPLOSS ┃ CANTIDAD ┃ RIESGO COLOCADO ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│          1 │ Calculada │ 27043.5 │    24000 │   0.0005 │             1.4 │
│          2 │ Calculada │   25000 │    24100 │   0.0005 │             0.4 │
└────────────┴───────────┴─────────┴──────────┴──────────┴─────────────────┘

SystemExit: 0

c:\Users\Agustin\Desktop\Zartex\BingX\venv\Lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
#  VERSION 6

import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import time
from http.client import HTTPException
import hmac
import hashlib
from rich.console import Console
from rich.table import Table
from datetime import datetime
import locale
import codecs



locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")

load_dotenv()

API_KEY = os.environ['API_KEY'] 
SECRET_KEY = os.environ['SECRET_KEY'] 

URL = 'https://open-api.bingx.com'

BINGX_ERRORS = {
    100001: "signature verification failed",
    100500: "Internal system error",
    80001: "request failed",
    80012: "service unavailable",
    80014: "Invalid parameter",
    80016: "Order does not exist",
    80017: "position does not exist"
}

services = {'GET_1':'/openApi/swap/v2/quote/contracts',
            'GET_2':'/openApi/swap/v2/user/balance',
            'GET_3':'/openApi/swap/v2/quote/price',
            'POST_1': '/openApi/swap/v2/trade/order',
}


def generate_signature(secret_key, query_params):
    """Generar firma encriptada"""
    signature = hmac.new(
        key = secret_key.encode('utf-8'), 
        msg = query_params.encode('utf-8'), 
        digestmod = hashlib.sha256
        ).hexdigest()
    return signature

def api_request(service, method='GET', query_params=None, sign=False):
    """docstring"""
    ROOT_URL = URL
    url = f'{ROOT_URL}{service}'

    header = {
        'Content-Type': 'application/json'
    }
    if sign:
        header.update({'X-BX-APIKEY': API_KEY})

    timestamp = int(time.time() *1000)
    params = f'timestamp={timestamp}'
    if query_params:
        params += f'&{query_params}'
    if sign:
        signature = generate_signature(SECRET_KEY, params)
        params += f'&signature={signature}'
    
    url += f'?{params}'
    if method == 'GET':
        response = requests.get(url, headers=header)
    else:
        response = requests.post(url, headers=header)

    if response.status_code != 200:
        print (f'status_code: {response.status_code}')
        print (f'message: {response.text}')
        raise HTTPException()

    response = response.json()
    if 'success' in response and not response.get('success'):
        print ('status_code=400')
        print (f'detail={BINGX_ERRORS.get(response.get("code"))}')
        raise HTTPException()

    return response


## SEGUNDA CAPA DE LA API

def get_account_balance():
    account_balance = api_request('/openApi/swap/v2/user/balance', method='GET', sign=True)
    return np.floor(float(account_balance['data']['balance']['balance']))

def price_BTC():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=BTC-USDT')['data']['price'])
def price_ETH():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=ETH-USDT')['data']['price'])
def price_XRP():
    return float(api_request('/openApi/swap/v2/quote/price', query_params='symbol=XRP-USDT')['data']['price'])


def obtener_contrato(par):
    contracts = api_request(services['GET_1'])
    contracts = pd.DataFrame(contracts['data'])
    if par == 'XRP':
        XRP_mask = contracts['symbol'].apply(lambda x: x.startswith('XRP')).values
        XRP_contract = contracts.loc[XRP_mask]
        return XRP_contract.iloc[0]
    elif par == 'BTC':
        BTC_mask = contracts['symbol'].apply(lambda x: x.startswith('BTC')).values
        BTC_contracts = contracts.loc[BTC_mask]
        BTC_contract = BTC_contracts.iloc[0]
        BTCD_contract = BTC_contracts.iloc[1]
        return BTC_contract
    return




## FUNCIONES DEL MAIN


def apalancamiento(precio_entrada, worst_sl, direccion_trade):
    """
    precio_entrada / apalancamiento = distancia entre el precio de entrada y el stop loss más alejado
    Para trabajar con margen de seguridad le quitamos una unidad al apalancamiento
    """
    dist = abs(precio_entrada - worst_sl)
    apalancamiento = int(precio_entrada / dist) -2
    if direccion_trade == 'LONG':
        precio_liquidacion = precio_entrada *(1 - 1/apalancamiento)
    elif direccion_trade == 'SHORT':
        precio_liquidacion = precio_entrada *(1 + 1/apalancamiento)
    return apalancamiento, precio_liquidacion

def generar_rows(n_entradas, estado_entradas, entradas, sls, qty_entradas):
    id = [str(i) for i in range(1, n_entradas+1)]
    estado_entradas = ['Calculada' if e else 'Omitida' for e in estado_entradas]
    entradas = [entradas[i] if i < len(entradas) else 0.0 for i in range(n_entradas)]
    sls = [sls[i] if i < len(sls) else 0.0 for i in range(n_entradas)]
    qty_entradas = [qty_entradas[i] if i < len(qty_entradas) else 0.0 for i in range(n_entradas)]
    riesgo = [abs(entradas[i] - sls[i])*qty_entradas[i] for i in range(n_entradas)]

    entradas = [str(round(elemento, price_precision))for elemento in entradas]
    sls = [str(round(elemento, price_precision))for elemento in sls]
    qty_entradas = [str(round(elemento, qty_precision))for elemento in qty_entradas]
    riesgo = [str(round(elemento, 1))for elemento in riesgo]
    return id, estado_entradas, entradas, sls, qty_entradas, riesgo

In [4]:
# VERSION 6: MAIN
if __name__ == '__main__':
    from tools.ingresar_datos import ingreso_bool, ingreso_bool_personalizado, entero_o_porcentual
    from itertools import count

    print ('''
    ============================================================
            BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
    ============================================================
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

    ''')


    # SETEO DE LAS VARIABLES INICIALES
    print ('Seleccione par a operar:')
    par = ingreso_bool_personalizado('BTC', 'XRP')
    print ('Obteniendo datos del contrato...')
    if not par:
        exit()

        
    contract = obtener_contrato(par)
    qty_precision = contract['quantityPrecision']
    price_precision = contract['pricePrecision']

    ## ESTABLECER EL APALANCAMIENTO MAXIMO DEL PAR (OBTENIDO MANUALMENTE INGRESANDO A LA PLATFORMA)#TODO: exportar esta data a un archivo externo
    if par == 'BTC':
        apalancamiento_maximo = 150
    elif par == 'XRP':
        apalancamiento_maximo = 125


    print ('Obteniendo datos de la cuenta...\n')
    # vol_cta = get_account_balance()#TODO: salir de la DEMO
    vol_cta = 50#TODO: borrar
    riesgo_op = 20#TODO: fijar a través de archivo externo
    print ('Direccion del trade:')
    direccion_trade = ingreso_bool_personalizado('LONG', 'SHORT')
    n_unidades = 1#TODO: fijar a través de archivo externo
    if n_unidades <= 0 or n_unidades > 5:
        raise ValueError('La posición solo admite hasta 5 entradas')

    # CALCULO DE LA POSICION
    vol_op = vol_cta * riesgo_op /100
    vol_unidad = vol_op / n_unidades

    display_data_inicial = """
>>    DATOS DE LA CUENTA
        - volumen cuenta  = {}

>>    DATOS DE LA POSICION
        - par = {}
        - riesgo posicion = {}%
        - direccion trade = {}
        - nº entradas = {}
        - volumen operacion = {}
        - volumen por entrada = {}

        """.format(vol_cta, par, riesgo_op, direccion_trade, n_unidades, vol_op, round(vol_unidad,2))
    print (display_data_inicial)

    print ('''
    ============================================================
                    MODULO DE DIVERSIFICACION
    ============================================================
    
    ''')
    estado_entradas = []
    for i in range(n_unidades):
        estado_entradas.append(ingreso_bool(f'Colocar entrada {i+1}?'))
    print (f'Entradas COLOCADAS: {sum(estado_entradas)}  |  Entradas ANULADAS: {len(estado_entradas)-sum(estado_entradas)}')


    target_entradas = []
    print ('\nCargando precio...')
    if par == 'BTC':
        benchmark = price_BTC()
    elif par == 'XRP':
        benchmark = price_XRP()
    print ('continuando\n')
    display_estado = """

    ============================================================
                            ENTRADA Nº {}
    ============================================================
    Precio de referencia {} = {}
    """
    entrada_count = count(1)
    for estado in estado_entradas:
        if estado:
            print (display_estado.format(next(entrada_count), par, benchmark))
            # DETERMINACION DE LA ENTRADA
            entrada = ingreso_bool_personalizado('MARKET', 'LIMIT', default='MARKET')
            if entrada == 'LIMIT':
                entrada, pct = entero_o_porcentual('>>  PRECIO DE ENTRADA (Vacío significa precio MARKET) ')
                if pct and direccion_trade=='LONG':
                    entrada = benchmark *(1-entrada)
                elif pct and direccion_trade=='SHORT':
                    entrada = benchmark *(1+entrada)
                elif not entrada:
                    print ('Ingresando orden LIMIT a precio MARKET')
                    entrada = benchmark
                    # raise ValueError('No se puede ingresar un orden LIMIT sin precio entrada')
            elif entrada == 'MARKET':
                entrada = benchmark
            else:
                raise ValueError('No se puede colocar una entrada vacia')
            # DETERMINACION DEL STOPLOSS
            sl, pct = entero_o_porcentual('>>  STOPLOSS ')
            if pct and direccion_trade=='LONG':
                sl = benchmark * (1 - sl)
            elif pct and direccion_trade=='SHORT':
                sl = benchmark * (1 + sl)
            elif not sl:#TODO meter la SL dentro de un bucle while o agregar la función de chance
                raise ValueError('No se puede puede ingresar una orden LIMIT sin target StopLoss')

            target_entradas.append((entrada, sl))
            # VERIFICACION
            for entrada, sl in target_entradas:
                if direccion_trade == 'LONG':
                    if entrada <= sl:
                        raise ValueError('En un trade LONG, la entrada no puede ser menor que el StopLoss')
                elif direccion_trade == 'SHORT':
                    if entrada >= sl:
                        raise ValueError('En un trade SHORT, la entrada no puede ser mayor que el StopLoss')


    # DIMENSIONAMIENTO DEL TRADE
    print ('\nCalculando posición...')
    #1. promediar entradas
    entradas = [entrada for entrada, _ in target_entradas]
    entrada_promedio = np.mean(entradas)
    #2. Obtener el stop loss más alejado
    sls = [sl for _, sl in target_entradas]
    if direccion_trade == 'LONG':
        worst_sl = np.min(sls)
    elif direccion_trade == 'SHORT':
        worst_sl = np.max(sls)
    #3. medir el apalancamiento para estar lo más cerca posible del stop loss más alejado, con un margen hardcoded de 2
    apal_x, precio_liq = apalancamiento(entrada_promedio, worst_sl, direccion_trade)

    if apal_x > apalancamiento_maximo:
        raise ValueError(f'El apalancamiento máximo para este par es de {apalancamiento_maximo}x\nEl apalancamiento calculado es de {apal_x}x')

    qty_entradas = [round(vol_unidad/abs(x[0] - x[1]),qty_precision) for x in target_entradas]




    display_resultado = """
    
    ============================================================
                            RESULTADOS
    ============================================================
    
    """


    # IMPRESION DE DESCRIPCION DE TRADE
    display_resultados =f"""
>>  DESCRIPCION TRADE {direccion_trade}
        - Precio de referencia = {benchmark}
        - Entrada promedio = {round(entrada_promedio, price_precision)}
        - StopLoss más alejado = {round(worst_sl, price_precision)}
        - Apalancamiento = {apal_x}
        - Precio de liquidación = {round(precio_liq, price_precision)}
        - Total comerciado = {sum([round(qty_e, qty_precision) for qty_e in qty_entradas])}
        - Pérdidas peor escenario = {round(sum([abs(entradas[i] - sls[i])*qty_entradas[i] for i in range(sum(estado_entradas))]), 2)}
        """
    print (display_resultados)

    # GENERACION DE TABLA DE RESUMEN DE ENTRADAS
    console = Console(record=True)
    table = Table(title="RESUMEN ENTRADAS: {} | PAR: {}".format(direccion_trade, par))

    table.add_column("Nº", justify="right", style="cyan", no_wrap=True)
    table.add_column("CONDICION", style="cyan")
    table.add_column("ENTRADA", justify="right", style="cyan")
    table.add_column("STOPLOSS", justify="right", style="cyan")
    table.add_column("CANTIDAD", justify="right", style="cyan")
    table.add_column("RIESGO", justify="right", style="cyan")

    row_1, row_2, row_3, row_4, row_5, row_6 = generar_rows(n_unidades, estado_entradas, entradas, sls, qty_entradas)

    for i in range(n_unidades):
        table.add_row(row_1[i], row_2[i], row_3[i], row_4[i], row_5[i], row_6[i])
    console.print(table)
    


    ######      ######      ######
    ##  GUARDAR POSICION EN TXT
    ######      ######      ######

    # CREAR CARPETA
    path = os.getcwd()
    file_location = 'registro'
    path = os.path.join(path, file_location)
    os.makedirs(path, exist_ok=True)

    # OBTENER DATA DE HOY
    fecha_actual = datetime.now()
    nombre_mes = fecha_actual.strftime("%B").title()
    hora = '\n\n\nTrade calculado el día {} a las {}'.format(fecha_actual.strftime("%d de {mes} de %Y").format(mes=nombre_mes), fecha_actual.strftime("%H:%M:%S"))
    
    # LEEMOS EL NOMBRE DE LOS FICHEROS DEL DIRECTORIO CREADO
    file_names = os.listdir(path=path)

    # SI LA LISTA ESTA VACIA CREAR EL PRIMER ARCHIVO
    if len(file_names) == 0:
        file_name = f'01_{direccion_trade}_{par}_{fecha_actual.strftime("%d_{}").format(nombre_mes)}.txt'

    # SI LA LISTA NO ESTA VACIA OBTENER EL ULTIMO ARCHIVO Y LEER EL CONTADOR
    else:
        last_file = file_names[-1]
        i = int(last_file.split('_')[0])
        file_name = f'{i+1:02d}_{direccion_trade}_{par}_{fecha_actual.strftime("%d_{}").format(nombre_mes)}.txt'


    file = os.path.join(path, file_name)


    with codecs.open(file, 'w', encoding='utf-8') as f:
        f.write('TradeGestorDEMO: v2 \nCalculadora de riesgo y gestor de posiciones\nExchange: BingX\n')
        f.write(display_data_inicial)
        f.write(display_resultados)
        f.write('\n\n\n')
        f.write(console.export_text())
        f.write(hora)
        f.write('\n   -- Desarrollado por Jackone Action Software Company\n   -- jackone.action.software@gmail.com')


    ### OBTENEER LOS VALORES DEL RESULTADO INDIVIDUAL DE LAS ENTRADAS Y DEL TRADE EN GENERAL

    import sys
    sys.exit(0)

    ## EJECUCION DE LA POSICION
    def ejecutar():
        """
        Función que ejecuta el trade
        """
        if input('Ejecutar?: '):
            print('Ejecutando...')
            return True
        else:
            print('Entrada pendiente')
            return False

    for entrada in target_entradas:
        if ejecutar():
            print('Entrada ejecutada')
        else:
            print('Entrada pendiente')




            BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

    
Seleccione par a operar:
< 1: BTC >    < 0: XRP >

Obteniendo datos del contrato...
Obteniendo datos de la cuenta...

Direccion del trade:
< 1: LONG >    < 0: SHORT >


>>    DATOS DE LA CUENTA
        - volumen cuenta  = 50

>>    DATOS DE LA POSICION
        - par = BTC
        - riesgo posicion = 20%
        - direccion trade = LONG
        - nº entradas = 1
        - volumen operacion = 10.0
        - volumen por entrada = 10.0

        

                    MODULO DE DIVERSIFICACION
    
    
Colocar entrada 1?
< 1: Si >    < 0: No >

Entradas COLOCADAS: 1  |  Entradas ANULADAS: 0

Cargando precio...
continuando



                            ENTRADA Nº 1
    Precio de referencia BTC = 26896.0
    
< 1: MARKET >    < 0: LIMIT >
>>  STOPLOSS 

Calculando posición...

>>  DESCRIPCION TRADE LONG
        - Precio de referencia = 26896.

             RESUMEN ENTRADAS: LONG | PAR: BTC             
┏━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┓
┃ Nº ┃ CONDICION ┃ ENTRADA ┃ STOPLOSS ┃ CANTIDAD ┃ RIESGO ┃
┡━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━┩
│  1 │ Calculada │ 26896.0 │  25551.2 │   0.0074 │   10.0 │
└────┴───────────┴─────────┴──────────┴──────────┴────────┘

SystemExit: 0

c:\Users\Agustin\Desktop\Zartex\BingX\venv\Lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
######## VIERNES 31/3

"""
>>    DATOS DE LA CUENTA
        - volumen cuenta  = 50.0
        - riesgo posicion = 40%

>>    DATOS DE LA OPERATIVA
        - par = BTC
        - nº entradas = 3
        - volumen operacion = 20.0
        - volumen por entrada = 6.67
"""

# El usuario ingresa 2: configuración app
# MODULO DE CONFIGURACION DE LA APP





# CONFIGURACION DE LA CUENTA
volumen_cuenta = 50.0
riesgo_op = 40

# CONFIGURACION DE LA OPERATIVA
par = 'BTC'
n_entradas = 3
vol_unidad = volumen_cuenta * riesgo_op / 100
vol_unidad = vol_unidad/n_entradas



## CONFIGUARCION DE LA APP


opciones = ['Configuración cuenta', 'Seleccionar cuenta', 'Activar contrato para otros pares', 'Volver']

from collections import deque
def create_options_enum(opciones:list) -> list:
        opciones_input = deque(range(len(opciones)), maxlen=len(range(len(opciones))))
        opciones_input.append('0')
        opciones_input = list(opciones_input)
        return list(map(lambda x: str(x), opciones_input))
## create_options_enum(opciones) ##OK

def create_options_display(opciones: list) -> str:
        opciones_display = ''
        opciones_enum = create_options_enum(opciones)
        for i, enum in enumerate(opciones_enum):
                opciones_display += f'{enum}. {opciones[i]}\n'
        return opciones_display
##print(create_options_display(opciones=opciones))#OK

def create_options_dict(opciones: list) -> dict:#TODO: Buscar implementación
        opciones_dict = {}
        opciones_enum = create_options_enum(opciones)
        for i, enum in enumerate(opciones_enum):
                opciones_dict[enum] = opciones[i]
        return opciones_dict


def activate_options(opciones: list) -> str:
        opciones_display = create_options_display(opciones)
        opciones_dict = create_options_dict(opciones)
        while True:
                print(opciones_display)
                opcion = input('Selecciona una opción: ')
                if opcion in opciones_dict:
                        return opcion
                else:
                        print('Opción no válida')


######## VIERNES 31/3




# abrir el archivo pickle que se llama "active.pkl"
import pickle

# check if file exists
if os.path.isfile('cuentas/active.pkl'):
        # I love you
        with open('cuentas/active.pkl', 'rb') as f:
                active = pickle.load(f)


active



# import pickle

# # Creamos una variable con un string vacío
# string_vacio = ""

# # Abrimos un archivo en modo de escritura binaria y escribimos la variable en el archivo
# with open("archivo.pickle", "wb") as archivo:
#     pickle.dump('Cuenta activada', archivo)

# # Cerramos el archivo
# archivo.close()

# # Abrimos el archivo en modo de lectura binaria y leemos la variable del archivo
# with open("archivo.pickle", "rb") as archivo:
#     string_vacio = pickle.load(archivo)

# # Imprimimos la variable para verificar que se haya leído correctamente
# print(string_vacio)









'--.. .- .-. - . -..- .--. .-. ---'

In [3]:
from tools.ingresar_datos import ingreso_bool, ingreso_bool_personalizado, entero_o_porcentual
def actualizar_contratos():
    """Se conecta a la API de BingX y descarga todos los contratos futuros disponibles en el disco local"""
    contracts = api_request(services['GET_1'])
    data = contracts['data']
    for i in range(len(data)):
        name_asset = contracts['data'][i]['asset']
        with open(f'contratos/{name_asset}.txt', 'w') as f:
            f.write(str(data[i]))
    
def cargar_contrato(par):
    name_contract = list(filter(lambda x: x.startswith(par), os.listdir('contratos')))[0]
    with open(f'contratos/{name_contract}', 'r') as f:
        contract = eval(f.read())
    return contract
    


In [3]:
## LUNES 3/4/2023
from tools.app_modules import cargar_contrato

par = 'LINK'
contract = cargar_contrato(par)
print (contract)

par = contract['asset']
symbol = contract['symbol']
qty_precision = contract['quantityPrecision']
price_precision = contract['pricePrecision']
max_leverage_l = contract['maxLongLeverage']
max_leverage_s = contract['maxShortLeverage']


{'contractId': '103', 'symbol': 'LINK-USDT', 'size': '0.1', 'quantityPrecision': 1, 'pricePrecision': 3, 'feeRate': 0.0005, 'tradeMinLimit': 1, 'maxLongLeverage': 75, 'maxShortLeverage': 75, 'currency': 'USDT', 'asset': 'LINK', 'status': 1}


{'contractId': '101',
 'symbol': 'ETH-USDT',
 'size': '0.01',
 'quantityPrecision': 2,
 'pricePrecision': 2,
 'feeRate': 0.0005,
 'tradeMinLimit': 1,
 'maxLongLeverage': 125,
 'maxShortLeverage': 125,
 'currency': 'USDT',
 'asset': 'ETH',
 'status': 1}

In [ ]:
#TODO: ingresar a cada uno de los contratos y verificar aquellos cuyo 'currency' sea distinto a 'USDT'


In [67]:
n_unidades = 5
estado_entradas = [True, True, False, False, False]
entradas = [24234, 234234]
sls = [22233, 22343]
qty_entradas = [0.0007, 0.0007]




id = list(range(1, n_unidades+1))
estado_entradas = ['Calculada' if e else 'Omitida' for e in estado_entradas]
entradas = [entradas[i] if i < len(entradas) else 0.0 for i in range(n_unidades)]
sls = [sls[i] if i < len(sls) else 0.0 for i in range(n_unidades)]
qty_entradas = [qty_entradas[i] if i < len(qty_entradas) else 0.0 for i in range(n_unidades)]
riesgo = [(entradas[i] - sls[i])*qty_entradas[i] for i in range(n_unidades)]




row_6


entradas

[24234, 234234, 0.0, 0.0, 0.0]

In [65]:
# sls
# entradas

# qty_entradas
# estado_entradas[len(estado_entradas)]
sum([precio*cantidad/apal_x for precio, cantidad in zip(entradas, qty_entradas)])#PERDIDAS

list(zip(entradas, qty_entradas))

[(27732.5, 0.0003004897983713453), (26500, 0.00031446540880503143)]

In [2]:
## COLOCACION DE ORDENES PARTE 1
## LEER EL DIRECTORIO CORRECTO Y SOLICITAR AL USUARIO QUE SELECCIONE EL ARCHIVO QUE QUIERE OPERAR

import os
import pandas as pd

def ultimas_ordenes(path='ordenes', ultimas=5):
    files = os.listdir(path)
    print ('ULTIMAS ORDENES:')
    posible_files = []
    for file in files:
        if not file.split('_')[0].isdigit():
            continue
        posible_files.append(file)
    for file in posible_files[-ultimas:]:
        print (file)
    print ('SELECCIONA el número de orden que quieres operar: default={}'.format(pd.Series(posible_files).max()))
    return posible_files


def seleccionar_orden(posible_files):
    ingreso = input('>> ')
    if ingreso == '':
        return pd.Series([int(num.split('_')[0]) for num in posible_files]).max()
    elif ingreso.isdigit():
        if ingreso[0] == '0':
            ingreso=ingreso[1:]
        if int(ingreso) > len(posible_files):
            print ('error')
            return seleccionar_orden(posible_files)
        return int(ingreso)
    else:
        print ('error')
        exit()

def buscar_orden(num_orden, posible_files):
    for file in posible_files:
        if int(file.split('_')[0]) == num_orden:
            return file
    print ('error')
    exit()


posible_files = ultimas_ordenes()
num_orden = seleccionar_orden(posible_files)
file = buscar_orden(num_orden, posible_files)
print (file)

ULTIMAS ORDENES:
01_SHORT_XRP_08_abril.txt
02_LONG_BTC_10_abril.txt
SELECCIONA el número de orden que quieres operar: default=02_LONG_BTC_10_abril.txt
02_LONG_BTC_10_abril.txt


In [3]:
## COLOCACION DE ORDENES PARTE 2
## DADO EL NOMBRE DEL ARCHIVO, ABRIR EL ARCHIVO Y LEER EL CONTENIDO

def leer_orden(file):
    path = os.path.join(os.getcwd(), 'ordenes', file)
    with open(path, 'r') as f:
        direccion_trade = f.readline().strip()
        contrato = eval(f.readline().strip())
        target_entradas = eval(f.readline().strip())
        apalancamiento = eval(f.readline().strip())
        monto_entrada = eval(f.readline().strip())
    return contrato, apalancamiento, direccion_trade, target_entradas, monto_entrada

contrato, apalancamiento, direccion_trade, target_entradas, monto_entrada = leer_orden(file)

In [10]:
## COLOCACION DE ORDENES PARTE 3
## CAMBIAR EL APALANCAMIENTO DE UN PAR

from tools.api_bingx import api_request, services

servicio = services['POST_1']
symbol = contrato['symbol']
direccion_trade
apalancamiento

query_params = f'symbol={symbol}&side={direccion_trade}&leverage={apalancamiento}'

response = api_request(service=servicio,
                        method='POST',
                        query_params=query_params,
                        header=True,
                        sign=True
                        )

headers -->  {'Content-Type': 'application/json', 'X-BX-APIKEY': 'qR7grfyuk1eenK4Ne2kIdVfpeaTB3Ki60TRCrUj0twIEw0nz9bTK19EmLbJ2OSXw2wuyaiWW3OYBGoKg'}
params -->  timestamp=1681146477584&symbol=BTC-USDT&side=LONG&leverage=150
url  -->  https://open-api.bingx.com/openApi/swap/v2/trade/leverage?timestamp=1681146477584&symbol=BTC-USDT&side=LONG&leverage=150&signature=2f7246f0240d4be4800949e5c0226cb5f29d30afb2d1104bcf83366cac1ac162
enviando un post method
SALIDA 3


In [5]:
## zartex web ada
preds = ["SI", "SI", "SI", "NO", "SI", "SI", "SI", "SI", "SI", "SI", "SI", "NO", "SI", "SI", "NO", "SI", "NO", "SI", "NO", "SI", "NO", "NO", "SI", "SI", "SI", "NO", "SI", "SI", "NO", "SI", "SI", "SI", "SI", "SI", "NO", "SI", "NO", "NO", "SI", "SI", "SI", "SI", "SI", "NO", "NO", "SI", "SI", "SI", "SI", "NO", "NO", "SI", "SI", "SI", "SI", "SI", "SI", "NO", "SI"]
check = [True if pred == "SI" else False for pred in preds]
aciertos = sum(check)
porc_aciertos = aciertos/len(preds) *100
result = "El porcentaje de aciertos es de {}%".format(round(porc_aciertos, 2))
print (result)

El porcentaje de aciertos es de 71.19%


In [1]:
### main 2: ok - VRSION 1


from tools.api_bingx_v2 import switch_leverage
from tools.app_modules import ultimas_ordenes, seleccionar_orden, buscar_orden, leer_orden

print ('''
============================================================
        BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
============================================================
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

''')

# INGRESAR ARCHIVO DE ORDENES

posible_files = ultimas_ordenes()
num_orden = seleccionar_orden(posible_files)
file = buscar_orden(num_orden, posible_files)


# OBTENER VALORES DE LA ORDEN

contrato, apalancamiento, direccion_trade, target_entradas, monto_entrada = leer_orden(file)
#TODO:
print ("symbol:{}, dirección: {}, apalancamiento: {}".format(contrato['symbol'], direccion_trade, apalancamiento))

# CAMBIAR EL APALANCAMIENTO DEL PAR

symbol = contrato['symbol']
response = switch_leverage(symbol, direccion_trade, apalancamiento)

if response['code'] == 0:
    print ('Actualización de apalancamiento OK: {}'.format(response['data']))
else:
    print ('Error al actualizar el apalancamiento: {}'.format(response['msg']))
    exit()




        BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT


ULTIMAS ORDENES:
01_SHORT_XRP_08_abril.txt
02_LONG_BTC_10_abril.txt
SELECCIONA el número de orden que quieres operar: default=02_LONG_BTC_10_abril.txt
Actualización de apalancamiento OK: {'leverage': 100, 'symbol': 'XRP-USDT'}


In [2]:
###  FUNCIONES NUEVAS PARA COLOCAR ORDENES

from tools.api_bingx_v2 import praseParam, send_request


def post_market_order(symbol, side, positionSide, quantity):
    "Colocar orden MARKET"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'MARKET',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)


def post_limit_order(symbol, side, positionSide, quantity, price):
    "Colocar orden LIMIT"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'LIMIT',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "price": price,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)


def post_trigger_limit_order(symbol, side, positionSide, quantity, price, stopPrice):
    "Colocar orden STOP-LIMIT"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'TRIGGER_LIMIT',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "price": price,
        "stopPrice": stopPrice,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)


def post_stop_market_order(symbol, side, positionSide, quantity,stopPrice):
    "Funcion para colocar orden de stop loss pendiente"
    "LIMIT-MARKET"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'STOP_MARKET',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "stopPrice": stopPrice,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)


def post_take_profit_market_order(symbol, side, positionSide, quantity,stopPrice):
    "Funcion para colocar orden de take-profit pendiente"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'TAKE_PROFIT_MARKET',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "stopPrice": stopPrice,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)


def post_trigger_market_order(symbol, side, positionSide, quantity,stopPrice):
    "STOP LIMIT para ejecutar orden MARKET"
    payload = {}
    path = '/openApi/swap/v2/trade/order'
    methed = "POST"
    paramsMap = {
        "symbol": symbol,
        "type": 'TRIGGER_MARKET',
        "side": side,
        "positionSide": positionSide,
        "quantity": quantity,
        "stopPrice": stopPrice,
        "timestamp": int(time.time() * 1000),
    }
    paramsStr = praseParam(paramsMap)
    return send_request(methed, path, paramsStr, payload)



# if type == "LIMIT":
#     paramsMap["price"] = price
# elif type == 'TRIGGER_LIMIT':
#     paramsMap["price"] = price
#     paramsMap["stopPrice"] = price
# elif type == 'STOP_MARKET' or type == 'TAKE_PROFIT_MARKET' or type == 'TRIGGER_MARKET':
#     paramsMap["stopPrice"] = stopPrice



In [1]:
from tools.api_bingx_v2 import switch_leverage
from tools.app_modules import ultimas_ordenes, seleccionar_orden, buscar_orden, leer_orden

print ('''
============================================================
        BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
============================================================
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT

''')

# INGRESAR ARCHIVO DE ORDENES

posible_files = ultimas_ordenes()
num_orden = seleccionar_orden(posible_files)
file = buscar_orden(num_orden, posible_files)



# OBTENER VALORES DE LA ORDEN POR ENTRADA

direccion_trade, contrato, target_entradas, apalancamiento, monto_entrada = leer_orden(file)


        BIENVENIDO A LA MEJOR CALCULADORA DE RIESGO
TradeGestorDEMO v2
Exchange: BingX
Cuenta: Future Perpetual
Pares: BTC-USDT  |  XRP-USDT


ULTIMAS ORDENES:
02_LONG_BTC_17_abril.txt
03_LONG_XRP_17_abril.txt
04_LONG_BTC_18_abril.txt
05_SHORT_BTC_19_abril.txt
06_SHORT_BTC_19_abril.txt
SELECCIONA el número de orden que quieres operar: 
default=06_SHORT_BTC_19_abril.txt


In [2]:
from tools.api_bingx import get_price
spread = 0.0005  #0.05%

symbol = contrato['symbol']
positionSide = direccion_trade

if positionSide == 'LONG':
    entrada_side = 'BUY'
    sl_side = 'SELL'
elif positionSide == 'SHORT':
    entrada_side = 'SELL'
    sl_side = 'BUY'

ordenes = []
for i in range(len(target_entradas)):
    entrada = {}
    sl = {}

    type_entrada = target_entradas[i][1]
    type_sl = 'TRIGGER_MARKET'#STOP_MARKET: promedia los stoploss

    if type_entrada=='TRIGGER_MARKET':
        benchmark = get_price(symbol)
        if positionSide == 'LONG':
            stopPrice = benchmark * (1 - spread)
        elif positionSide == 'SHORT':
            stopPrice = benchmark * (1 + spread)

    quantity = monto_entrada[i]
    
    orden = {
        "symbol": symbol,
        "positionSide": positionSide,
        'quantity': quantity
    }

    entrada.update(orden)
    entrada.update({'side': entrada_side,'type': type_entrada})
    sl.update(orden)
    sl.update({'side': sl_side,'type': type_sl, 'stopPrice': target_entradas[i][-1]})

    if type_entrada.find('LIMIT') != -1:
        entrada.update({'price': target_entradas[i][0]})
    elif type_entrada.find('TRIGGER') != -1:
        entrada.update({'stopPrice':stopPrice})

    ordenes.append((entrada, sl))

In [3]:
entrada = ordenes[0][0]
entrada

{'symbol': 'BTC-USDT',
 'positionSide': 'SHORT',
 'quantity': 0.0053,
 'side': 'SELL',
 'type': 'MARKET'}

In [4]:
from tools.api_bingx_v2 import post_order

# CAMBIAR EL APALANCAMIENTO DEL PAR
response = switch_leverage(symbol, direccion_trade, apalancamiento)

if response['code'] == 0:
    print ('Actualización de apalancamiento OK: {}'.format(response['data']))
else:
    print ('Error al actualizar el apalancamiento: {}'.format(response['msg']))
    exit()

response = post_order(**entrada)

Actualización de apalancamiento OK: {'leverage': 29, 'symbol': 'BTC-USDT'}


In [6]:
sl = ordenes[0][1]
sl

response = post_order(**sl)

In [7]:
response

{'code': 0,
 'msg': '',
 'data': {'order': {'symbol': 'BTC-USDT',
   'orderId': 1648703245859684352,
   'side': 'BUY',
   'positionSide': 'SHORT',
   'type': 'STOP_MARKET'}}}

In [24]:




# CAMBIAR EL APALANCAMIENTO DEL PAR
response = switch_leverage(symbol, direccion_trade, apalancamiento)

if response['code'] == 0:
    print ('Actualización de apalancamiento OK: {}'.format(response['data']))
else:
    print ('Error al actualizar el apalancamiento: {}'.format(response['msg']))
    exit()


# COLOCACIÓN DE LAS ORDENES DE ENTRADA
print('# COLOCACIÓN DE LAS ORDENES DE ENTRADA')


NameError: name 'sl' is not defined

In [2]:
nombre_variables = list(globals().keys())

In [12]:
nombre_variables[-25:]

"""
    'direccion_trade': str -> LONG | SHORT
    'target_entradas': list[tuple] -> ordenes
    'apalancamiento': int
    'monto_entrada': list -> quantity
    'symbol': str
    'response':
    'n_entrada': int
"""

['__',
 '___',
 '__vsc_ipynb_file__',
 '_i',
 '_ii',
 '_iii',
 '_i1',
 'switch_leverage',
 'post_market_order',
 'ultimas_ordenes',
 'seleccionar_orden',
 'buscar_orden',
 'leer_orden',
 'posible_files',
 'num_orden',
 'file',
 'direccion_trade',
 'contrato',
 'target_entradas',
 'apalancamiento',
 'monto_entrada',
 'symbol',
 'response',
 'n_entrada',
 '_i2']